## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Hello There In this notebokk we are going to scrape the Full time and part time jobs data for the data related jobs like:<br>
Data analyst, ML dev, Data scientist, Data engineer etc ..<br><br>

**Important note:**
We will analyze some countries only for indeed becuase indeed needs to specify which country to look at.

## <center><strong>Importing <span style= 'color: #48e0dc'>Packeges

In [34]:
import re
import json
import time
import scrapy
import random
import pickle
import sqlite3
import warnings
import requests
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import count
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from scrapy.http import HtmlResponse
from scrapy.crawler import CrawlerProcess
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from IPython.display import set_matplotlib_formats
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [35]:
%matplotlib inline
COLORS = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']
MAX_PAGES = 300

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings('ignore')

ip_port_list = [('75.84.199.80', '80'),
                ('172.173.241.207', '80'),
                ('209.145.60.213', '80'),
                ('86.109.3.28', '80'),
                ('65.111.241.211', '80'),
                ('144.34.162.125', '80')]

## <center><strong>Setting up the<span style= 'color: #5cd3f7'> web scrapers

In [36]:
data_jobs_titles = ['Data entry', 'Data engineer',
                    'Data scientist', 'Data analyst',
                    'ML developer']

indeed_countries = ['de', 'uk']#, 'www'] USA don't have to be spesfied in the URL so we will use 'www' instead
linkedin_countries = ['European Union', 'United States']
ip_port = random.choice(ip_port_list)

In [37]:
import cloudscraper 

# I will use it for when i need to scrape a lot of data fast.
def scrape_page_fast(url: str) -> BeautifulSoup:

    scraper = cloudscraper.create_scraper(delay=10, browser="chrome") 
    content = scraper.get(url).text 

    return BeautifulSoup(content)



def scrape_page(url: str, retrieve_new_url= False) -> BeautifulSoup:
    
    driver = webdriver.Chrome()
    driver.get(url)
    
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    html = driver.page_source
    
    if retrieve_new_url:
        new_url = driver.current_url
        
        driver.quit()
        return BeautifulSoup(html), new_url
        
    driver.quit()
    return BeautifulSoup(html)

In [44]:
# Comparing the two scrapers speed.
%timeit scrape_page_fast('https://linkedin.com')
%timeit scrape_page('https://linkedin.com')

# We can find that there's about 10X speed for the instant scraper 🤯.

1.41 s ± 480 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
13.3 s ± 2.43 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
# def indeed_scraper(page: int,
#                    country: str,
#                    job_title: str) -> BeautifulSoup:

#     modefid_page = (page - 1) * 10

#     data_job_title = job_title.replace(' ', '+')

#     # Pages in indeed.com are zero based indexed
#     url: str = f'https://{country}.indeed.com/jobs?q={data_job_title}&sc=0kf%3Aattr%28DSQF7%29%3B&start={modefid_page}' + \
#     '&pp=gQClAAAAAAAAAAAAAAACCXklWwCSAQIBK7YBBwDUbujiRGBahYW5TppjdUz7DjXn3aPZSbT47IoJ5LLbuzpYcXwZdzJ6rKHf' + \
#     '6gPWFkXVGxKKGxW-JAKb8BFo_hZAkBd7trBBTY32J2CrOuA3V9dGD_bre-lArmi9DRYlcah6hvoRfsYUNYSoQwIa8VOMZMxvH-s2Dlh' + \
#     'UPvUP-_Dz9ls4i-OLqVGpGh4AAA&vjk=b997bb0dddadea'

#     soup = scrape_page(url)
#     loaded_page = int(soup.find('button', 'css-ns2mzi e8ju0x51').text)

#     # available_pages = []

#     # for button in soup.find_all('button', 'css-1qt7hdn e8ju0x50'):
#     #     available_pages.append(int(button.text))

#     if (int(page) != loaded_page):
#         # We will use KeyError as standard to represent getting out of the max pages.
#         raise KeyError
        
#     print(f'Page {page} Loaded successfully.')

#     return  soup

In [39]:
def linkedin_scraper(country:   str, # This scraper scrapes all pages at once
                     job_title: str,
                     page:      int) -> BeautifulSoup:

    page: int = (page - 1) * 25
    url: str = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={job_title}&' + \
    f'location={country}&geoId=90000084&trk=public_jobs_j%20obs-search-bar_search-submit&position=1&pageNum=0&start={page}'

    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')

    return  soup

## <center><strong>Collecting<span style= 'color: #5cd3f7'> jobs </span> data

**What we will do in this section:**

1. Collect all pages we need from each scraper and stack them together as HTML code.
2. Scrape for jobs cards links from linkedin and indeed.
3. Scrape for the data in those links and store the data in a DataFrame
4. We may also consider collecting total results per each platform, country etc ..

#### **Stack pages together as HTML code**

In [40]:
def convert_to_integer(number_string):

    cleaned_string = ''.join(filter(str.isdigit, number_string))
    cleaned_string = cleaned_string.rstrip('+')
    integer_value = int(cleaned_string)

    return integer_value

In [12]:
soups = {'soup': [],
         'job_title': [],
         'country': [],
         'platform': [],
         'total_jobs': []}

flag: bool = False
max_page: int = 4 # TOREMVE

# Linked In
print('Starting LinkedIn scraper.')
for country in linkedin_countries:

    print(f'Loading the {country} data.')
    for job_title in data_jobs_titles:
        
        full_soup: str = ''
        total_jobs = convert_to_integer(scrape_page(f'https://www.linkedin.com/jobs/search?keywords={job_title}&location={country}&' + 
                                                    f'\geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0').find(
                                                        'span', 'results-context-header__job-count').text)
        
        for i in count(0):
            page = i + 1
            page_soup: BeautifulSoup = linkedin_scraper(country= country,
                                                        job_title= job_title,
                                                        page= i + page)
            full_soup += str(page_soup)
            full_soup += ' <br> '

            if (page_soup.find_all('li') is None) or (page == max_page):
                break

        soups['platform'].append('LinkedIn')
        soups['total_jobs'].append(total_jobs) 
        soups['soup'].append(full_soup)
        soups['job_title'].append(job_title)
        soups['country'].append(country)

        print(f'Finished loading {job_title} jobs from the {country}.')
    print('\n')

Starting LinkedIn scraper.
Loading the European Union data.
Finished loading Data entry jobs from the European Union.
Finished loading Data engineer jobs from the European Union.
Finished loading Data scientist jobs from the European Union.
Finished loading Data analyst jobs from the European Union.
Finished loading ML developer jobs from the European Union.


Loading the United States data.
Finished loading Data entry jobs from the United States.
Finished loading Data engineer jobs from the United States.
Finished loading Data scientist jobs from the United States.
Finished loading Data analyst jobs from the United States.
Finished loading ML developer jobs from the United States.




In [18]:
# Indeed scraping TODO
# for country in indeed_countries:
#     for job_title in data_jobs_titles:
#         stacked_pages_soup: str = ''

#         for i in count(0):

#             try:
#                 page = i + 1
#                 soup: BeautifulSoup = indeed_scraper(country= country,
#                                                      job_title= job_title,
#                                                      page= page)

#                 stacked_pages_soup += '\n<br> ' + str(soup)
#                 if page > max_page:
#                     print('Finished loading current country or job title.\n')
#                     break

#             except KeyError as e:
#                 print('Finished loading current country or job title.\n')
#                 break

#         soups['platform'].append('Indeed')
#         soups['soup'].append(stacked_pages_soup)
#         soups['job_title'].append(job_title)
#         soups['country'].append(country)

#### **Collecting jobs & companies basic data**

In [19]:
def remove_escape_sequences(strings):

    new_strings = []
    escapes = ''.join([chr(char) for char in range(1, 32)])
    translator = str.maketrans('', '', escapes)

    for string in strings:
        new_string = string.translate(translator)
        new_strings.append(new_string)
        
    return tuple(new_strings)

In [20]:
soups = pd.DataFrame(soups)

soups['jobs_locations'] = np.nan
soups['listing_dates'] = np.nan
soups['jobs_titles'] = np.nan
soups['companies_names'] = np.nan

soups['companies_links'] = np.nan
soups['jobs_links'] = np.nan


# Collecting linkedIn jobs links
for i, soup in enumerate(soups[soups['platform'] == 'LinkedIn']['soup']):

    soup = BeautifulSoup(soup)

    linkedin_jobs_locations = tuple(
        [element.text for element in soup.find_all('span', 'job-search-card__location')])

    linkedin_jobs_listing_dates = tuple(
        [element['datetime'] for element in soup.find_all('time', 'job-search-card__listdate')])

    linkedin_jobs_titles = tuple(
        [element.text for element in soup.find_all('h3', 'base-search-card__title')])

    linkedin_jobs_companies_names = tuple(
        [element.text for element in soup.find_all('a', 'hidden-nested-link')])

    
    linkedin_jobs_links = tuple(
        [element['href'] for element in soup.find_all('a', 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')])
    
    linkedin_companies_links = tuple(
        [element['href'] for element in soup.find_all('a', 'hidden-nested-link')])

    # I will use sets and tuples instead of lists because they are faster.
    soups['jobs_locations'][i] = remove_escape_sequences(linkedin_jobs_locations)
    soups['listing_dates'][i] = linkedin_jobs_listing_dates
    soups['jobs_titles'][i] = remove_escape_sequences(linkedin_jobs_titles)
    soups['companies_names'][i] = remove_escape_sequences(linkedin_jobs_companies_names)

    soups['jobs_links'][i] = linkedin_jobs_links
    soups['companies_links'][i] = linkedin_companies_links

In [21]:
# Collecting indeed jobs links TOFIX
# for i, soup in enumerate(soups[soups['platform'] == 'Indeed']['soup']):
    
#     indeed_jobs_links: list = []
#     indeed_companies_links: list = []

#     for job in BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
#         job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job['data-jk'])
#         indeed_jobs_links.append(job_link)

#     for job_link in indeed_jobs_links[:5]: # WE WILL REMOVE THIS LIMIT
#         company_link = BeautifulSoup(scrape_page(job_link)).find('a', 'css-775knl emf9s7v0')
#         indeed_companies_links.append(company_link)
#         print(company_link)
        
#     soups['total_jobs'][i] = len(indeed_jobs_links)
#     soups['jobs_links'][i] = indeed_jobs_links
#     soups['companies_links'][i] = indeed_companies_links

In [25]:

soups.head()

,soup,job_title,country,platform,total_jobs,jobs_locations,listing_dates,jobs_titles,companies_names,companies_links,jobs_links
0,"\n<li>\n<div class=""base-card relative w-full ...",Data entry,European Union,LinkedIn,258000,"( Fremont, CA , ...","(2023-07-14, 2023-07-14, 2023-07-14, 2023-06-1...",( Entry Level - Data Entry ...,( Get.It Recruit - Administrative ...,(https://www.linkedin.com/company/get-it-recru...,(https://www.linkedin.com/jobs/view/entry-leve...
1,"\n<li>\n<div class=""base-card relative w-full ...",Data engineer,European Union,LinkedIn,40000,"( San Francisco, CA , ...","(2023-04-07, 2023-06-15, 2023-04-28, 2023-05-2...",( Data Engineer ...,"( Patreon , Sh...",(https://www.linkedin.com/company/patreon?trk=...,(https://www.linkedin.com/jobs/view/data-engin...
2,"\n<li>\n<div class=""base-card relative w-full ...",Data scientist,European Union,LinkedIn,149000,"( San Francisco, CA , ...","(2023-06-16, 2023-07-10, 2023-05-01, 2023-07-0...",( Data Scientist ...,"( Experfy , Gl...",(https://www.linkedin.com/company/experfy?trk=...,(https://www.linkedin.com/jobs/view/data-scien...
3,"\n<li>\n<div class=""base-card relative w-full ...",Data analyst,European Union,LinkedIn,158000,"( Pleasanton, CA , ...","(2023-06-19, 2023-02-20, 2023-05-11, 2023-05-0...",( Entry Level Data Analyst ...,( Flexon Technologies Inc. ...,(https://www.linkedin.com/company/flexon-techn...,(https://www.linkedin.com/jobs/view/entry-leve...
4,"\n<li>\n<div class=""base-card relative w-full ...",ML developer,European Union,LinkedIn,994,"( San Francisco, CA , ...","(2023-06-05, 2023-06-29, 2023-07-05, 2023-05-1...",( Engineer (Full Stack / ML...,"( Sieve , Auro...",(https://www.linkedin.com/company/sievedata?tr...,(https://www.linkedin.com/jobs/view/engineer-f...


#### **Loading more jobs data**

In [ ]:
# Collecting more LinkedIn data

# We will collect the: 
#    describtion, credential category, location type
#
soups['describtion'] = np.nan
soups['credential_category'] = np.nan
soups['location_type'] = np.nan
soups['employment_type'] = np.nan
soups['industry'] = np.nan
soups['country'] = np.nan

for i, jobs in enumerate([soups['jobs_links'][0]]): # TOREMOVE remove the loop limit

    describtion:           list = []
    credential_category:   list = []
    location_type:         list = []
    employment_type:       list = []
    industry:              list = []
    country:               list = []
    
    for job in jobs:
            
        try:
            job_data:  dict = json.loads(scrape_page_fast(job).find(
                'script', attrs= {'type': 'application/ld+json'}).text)
        except:
            continue
            
        try: job_data['jobLocationType'] is True
        except: job_data['jobLocationType'] = np.nan
            
        try: job_data['applicantLocationRequirements'] is True
        except: job_data['applicantLocationRequirements'] = {'name': np.nan}

        try: job_data['educationRequirements'] is True
        except: job_data['educationRequirements'] = {'credentialCategory': np.nan}
        
        describtion               .append(job_data['description'])
        credential_category       .append(job_data['educationRequirements']['credentialCategory'])
        location_type             .append(job_data['jobLocationType'])
        employment_type           .append(job_data['employmentType'])
        industry                  .append(job_data['industry'])
        country                   .append(job_data['applicantLocationRequirements']['name'])

    # We will use tuples becuase they are faster and more memory effecient.
    soups['describtion']          [i] = tuple(describtion)
    soups['credential_category']  [i] = tuple(credential_category)
    soups['location_type']        [i] = tuple(location_type)
    soups['employment_type']      [i] = tuple(employment_type)
    soups['industry']             [i] = tuple(industry)
    soups['country']              [i] = tuple(country)

In [47]:
# TODO (loading indeed data.)

#### **Loading more companies data**